In [1]:
from ccnet_spark.text_normalizer import normalize
from ccnet_spark.pipe_preprocess import load_segments
from ccnet_spark.pipe_hash import compute_hashes,split_doc2para
from ccnet_spark.pipe_lid import predictLang
from ccnet_spark.pipe_tokenized import doSentencePiece
from ccnet_spark.pipe_perplexity import doDocLM
from ccnet_spark.pipe_ppbucket import doPPBucket
from ccnet_spark.pipe_save import save_partation,load_partation,load_all
import time
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import explode
from pyspark.sql.functions import sum as spark_sum

# 初始化 SparkSession
spark = SparkSession.builder.appName("CCNETSpark")  \
                    .config("spark.executor.memory", "64g") \
                    .config("spark.driver.memory", "32g") \
                    .config("spark.driver.maxResultSize", "32g") \
                    .config('spark.sql.execution.arrow.pyspark.enabled', 'true') \
                    .getOrCreate()

24/04/03 15:48:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
def getModePara(mode):
    if(mode=="test"):
        n_segments=10
        cache_folder="/root/wxl_folder/cache_data/"
        date="2019-09" ## hardcode ,现在只能是这个
        segments=[i for i in range(n_segments)]
        min_len=300
        isSample=True
        sampleRate=0.01
        num_partitions=1
    else:
        n_segments=4
        cache_folder="/root/wxl_folder/cache_data/"
        date="2019-09" ## hardcode ,现在只能是这个
        segments=[i for i in range(n_segments)]
        min_len=300
        isSample=False
        sampleRate=1
        num_partitions=4
    return [cache_folder,date,segments,min_len,isSample,sampleRate,num_partitions]
mode="dev"
cache_folder,date,segments,min_len,isSample,sampleRate,num_partitions=getModePara(mode)

In [16]:
selected_bucket="middle"
selected_lang="en"
df_p=load_partation(spark,selected_lang,selected_bucket,cache_folder,date,isSample,sampleRate,min_len)
res=df_p.filter(df_p.cc_segment == "1").select("url").collect()
sorted_res = sorted(res, key=lambda x: x.url)
print(len(sorted_res))
for url in sorted_res:
    print(url.url)

2590
http://100words.com/entry/54352
http://1015fm.com.au/category/community/page/156/
http://134.155.100.214/members.html
http://3000toys.blogspot.com/2014/
http://36poses.org/new-information-about-cold-sores-and-herpes.html
http://3dprintingforum.us/viewforum.php?f=33&sid=8e90acfb12b1b318ab96f60dfcffc44c
http://585437.xobor.com/t1307f18-Hugs-Application-1.html
http://7criminalminds.blogspot.com/2009/08/
http://80breakfasts.com/double-chocolate-brownie-bites/
http://88y6897.com/topic/Adolf-Hitler
http://8mmforum.film-tech.com/cgi-bin/ubb/ultimatebb.cgi?ubb=get_topic;f=1;t=003485
http://8mmforum.film-tech.com/cgi-bin/ubb/ultimatebb.cgi?ubb=get_topic;f=6;t=002346
http://960kgkl.com/this-car-color-insurance-myth-is-getting-busted-once-and-for-all/
http://academiclearninglabs.com/team-view/gail-hodel/
http://ace-ed.org.uk/advice-about-education-for-parents/special_educational_needs/asking-for-an-ehc-needs-assessment?f57c55e1-710c-48c1-95f2-7a39eceae409
http://addicted-to-stamps.blogspot.c

In [6]:
df=load_all(spark,cache_folder,date,isSample,sampleRate,min_len)

In [11]:
df.filter(df.cc_segment==3).count()

41271